# Comparing degenerations of spinor varieties

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

In this notebook accompanying our paper "Regular subdivisions, bounds on initial ideals, and categorical limits", we compute degenerations of spinor varieties. We show that for spinor varieties of Type $D_5$ and $D_6$, we have that $\mathrm{in}_w(I) = I_w$, where $I_w$ is the lower bound as defined in our paper. 

We refer to `Computing_I_w_and_Omega.ipynb` for a thorough explanation and documentation of the lower bound ideal $I_w$ and the computation of the fan $\Omega(I) = \{w \in \mathbb{R}^n | \mathrm{in}_w(I) = I_w\}$, whereas this notebook only covers the extended example computation for spinor varieties. 
    

In [ ]:
using Oscar;
using Combinatorics;
pm = Polymake;

include("InitialIdealsRegularSubdivisions.jl")
include("gfan_functions.jl")



## Spinor varieties


The space of totally isotropic subspaces of a $2n$-dimensional vector space equipped with a symmetric, nondegenerate form of Witt index $n$ is an algebraic variety which consists of two irreducible components that are projectively equivalent to each other. The _spinor variety_ $\mathbb{S}_{n}$ is one of these irreducible components. It is a projective variety via the Wick embedding $J_n\subset S$ where $S$ is a polynomial ring with a generator for each subset of $[1,n]$ that has the same number of elements as $n$ modulo $2$. In [Cor], it is shown that $(J_n^{\circ})_w = \mathrm{in}_{w}(J_n)^{\circ}$ for $n\leq 5$ and $w\in \mathrm{Trop}(J_n)$. Here, we demonstrate the equality $(J_n)_w = \mathrm{in}_{w}(J_n)$ for $n\leq 5$ and $w\in \mathrm{Trop}(J_n)$.

---

**Function:** `spinor_variety_D(n)`

*Description*: For an integer `n` computes the ideal generating the Spinor variety of type $D_n$.

In [ ]:
function spinor_variety_D(n)
n = n-1
l(i, I) = length([j for j in I if j < i])
Le = vcat([reverse.(sort(reverse.(subsets(n+1, k)))) for k in 0:2:n+1]...)
Lo = vcat([subsets(n+1, k) for k in 1:2:n+1]...)
RDe, xDe = graded_polynomial_ring(QQ, :xDe => Le)
xDe = Dict(k => v for (k, v) in zip(Le, xDe))

relsDe = []
for (I, J) in subsets(Lo, 2)
    SD = symdiff(I, J)
    if length(SD) >= 4
        push!(relsDe, sum([(-1)^(l(i, I) + l(i, J))*xDe[sort(symdiff(I, i))]*xDe[sort(symdiff(J, i))] for i in SD]))
    end
end
return ideal(relsDe)
end

*Example*: We compute ideals of the spinor varieties of type $D_5$ and $D_6$.

In [ ]:
I = spinor_variety_D(5)

J = spinor_variety_D(6);

## The case $n = 5 $

In this section we focus on the Spinor variety associated to $D_5$, whose ideal `I` we computed in the previous section. 

We briefly recall the definitions of $I_w$ and $\Omega(I)$. Let $\Delta = \mathcal{A}(I)$ be the point configuration of $I$. A vector $w\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{w}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{w}(\Delta)\}$$

and the ideal they generate is

$$I_{w} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

For an ideal $I$ with point configuration $\Delta$, we define $\Omega(I)$ by

$$ \Omega(I) = \{w \in \mathbb{R}^{n} \, : \, \mathrm{in}_{w} I = I_{w} \}. $$

This set is the support of a subfan of the secondary fan $\mathrm{Sec}(\Delta)$.

---

To show that $\mathrm{in}_w(I) = I_w$ for all weight vectors $w$, we compute $\mathrm{Trop}(I)$ using `tropical_variety`. For each maximal cone in  $\mathrm{Trop}(I)$, we obtain a weight $w$ in the relative interior, and compare the initial degeneration $\mathrm{in}_w(I)$ (computed via `initial`) to the ideal $I_w$ we compute using `ideal_w` as documented in `Computing_I_w_and_Omega.ipynb`. In the process, we count whenever equality is achieved. We then compare this number to the number of maximal cells of $\mathrm{Trop}(I)$, and obtain equality.
 

In [ ]:
RR, y = polynomial_ring(QQ, length(gens(base_ring(I)), :y);
g = hom(base_ring(I), RR, gens(RR));
II = g(I);
TV = tropical_variety(II)[1];
println(length(maximal_polyhedra(TV)))

c = 0;
inIs = Set([]);
for C in maximal_polyhedra(TV)
    w = sum(rays_modulo_lineality(C)[1])
    w = lcm(denominator.(w))*w
    
    inI = initial(I, tropical_semiring_map(QQ, max), w)
    push!(inIs, inI)
    
    c += ideal_w(Ifour, -w) == inI
end
println(c, "\n", length(inIs))

## The case $n = 6$ 

We now turn to the Spinor variety associated to $D_6$, given by the ideal `J` computed above. Similar to the previous section, we wish to show that $\mathrm{in}_w(J) = J_w$ for all $w \in \mathrm{Trop}(J)$. On the whole, we proceed very similarly to the case $n=5$, however, as the computation of the tropical variety takes a while, we load it in pre-computed using the function `parse_tropical_file` which we provide in the supplementary file `gfan_functions.jl`. 

Afterwards, we proceed analogously to the above:  For each maximal cone in  $\mathrm{Trop}(J)$, we obtain a weight $w$ in the relative interior, and compare the initial degeneration $\mathrm{in}_w(J)$ (computed via `initial`) to the ideal $J_w$ we compute using `ideal_w` as documented in `Computing_I_w_and_Omega.ipynb`. In the process, we count whenever equality is achieved. We then compare this number to the number of maximal cells of $\mathrm{Trop}(J)$, and obtain equality. Again, for the sake of running time, this notebook only performs the comparison for the last 1000 maximal cones in the tropical variety.

In [ ]:

rays, M = parse_tropical_file("precomputed_data/n6trop.txt", true);

c = 0
inJs = Set([])
for i in nrows(M) - 1000 : nrows(M)
    R = [r for (j, r) in enumerate(rays) if M[i, j]]
    w = sum(R)
    inJ = initial(J, tropical_semiring_map(QQ, max), w)
    
    Jw = ideal_w(J, -w)
    c += Jw == inJ
    
    push!(inJs, inJ)
end
c
